# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [6]:
spark = SparkSession.builder.appName('Regression').getOrCreate()


In [7]:
data=spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [8]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [9]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [13]:
assembler = VectorAssembler(
    inputCols=["Age", "Tonnage", 
               "passengers",'length','cabins','passenger_density'],
    outputCol="features")

In [14]:
output = assembler.transform(data)

In [15]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|[17.0,101.353,26....|
|    Ecstasy|   Carnival| 22|            70.367|     20.

In [16]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [17]:
final_data = output.select("features",'crew')

In [18]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [19]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

In [20]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [21]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.01101051005240223,0.003750074463217927,-0.1142722142357283,0.3340878172061581,0.8442386472104478,0.011417608570565771] Intercept: -0.8407386162492645


In [22]:
test_results = lrModel.evaluate(test_data)

In [23]:
test_results.r2

0.9278831398410272

In [24]:
test_results.rootMeanSquaredError

0.8172224692221216

In [25]:
unlabeled_data = test_data.select('features')

In [26]:
predictions = lrModel.transform(unlabeled_data)

In [27]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,93.0,23.94,9...| 10.47933657582722|
|[6.0,112.0,38.0,9...|11.348094296720907|
|[6.0,113.0,37.82,...|11.858310618170444|
|[6.0,158.0,43.7,1...|14.059658645792425|
|[7.0,116.0,31.0,9...|12.723975601217429|
|[8.0,91.0,22.44,9...|10.007454677311697|
|[9.0,85.0,19.68,9...| 9.054201840084627|
|[9.0,88.5,21.24,9...| 9.623758415112766|
|[9.0,105.0,27.2,8...|11.207697704837507|
|[9.0,110.0,29.74,...|12.230901377709738|
|[9.0,113.0,26.74,...|11.375440191985726|
|[9.0,116.0,26.0,9...|11.185829110668488|
|[10.0,91.62700000...| 9.220278242734851|
|[10.0,151.4,26.2,...|10.638349493415008|
|[11.0,58.6,15.66,...| 7.255575517762434|
|[11.0,86.0,21.24,...|  9.57888943073665|
|[11.0,90.09,25.01...| 9.007734761474772|
|[11.0,91.0,20.32,...|  9.02394591418837|
|[11.0,138.0,31.14...|13.055739182214767|
|[12.0,25.0,3.88,5...|3.0454746979783165|
+--------------------+------------

In [30]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 0.8172224692221216
MSE: 0.6678525642015015
